In [1]:
import pandas as pd  
import numpy as np  
from geopy.distance import distance
import geopandas as gpd
import requests

In [2]:
utd19_path = '../../simulator_appendix/datasets/utd19_u.csv'
df = pd.read_csv(utd19_path)

/var/folders/0h/f5d7gkhd32v8pr7h9szg79wc0000gn/T/ipykernel_7357/3968526326.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(utd19_path)


In [3]:
df.head()

,day,interval,detid,flow,occ,error,city,speed
0,2017-05-06,0,06.X-2li,12.0,0.0,1.0,augsburg,NaN
1,2017-05-06,300,06.X-2li,12.0,0.0,1.0,augsburg,NaN
2,2017-05-06,600,06.X-2li,12.0,0.0,1.0,augsburg,NaN
3,2017-05-06,900,06.X-2li,16.0,0.0,1.0,augsburg,NaN
4,2017-05-06,1200,06.X-2li,16.0,0.0,1.0,augsburg,NaN


In [4]:
df[df['city'] == 'paris'].head()

,day,interval,detid,flow,occ,error,city,speed
115990908,2016-01-01,3600,1,2340.0,0.053,NaN,paris,NaN
115990909,2016-01-01,7200,1,2460.0,0.053,NaN,paris,NaN
115990910,2016-01-01,10800,1,2226.0,0.050,NaN,paris,NaN
115990911,2016-01-01,14400,1,1761.0,0.038,NaN,paris,NaN
115990912,2016-01-01,18000,1,1465.5,0.031,NaN,paris,NaN


In [5]:
df_paris_flow = df[df['city'] == 'paris']

In [6]:
print('Paris data size:', len(df_paris_flow), 'entries')

Paris data size: 1548314 entries


In [7]:
print('Date interval:', df_paris_flow['day'].min(), 'to', df_paris_flow['day'].max())

Date interval: 2016-01-01 to 2016-12-01


In [8]:
print('Number of detid:', len(df_paris_flow['detid'].unique()))

Number of detid: 209


In [ ]:
#let's normalize the data
df_paris_flow['flow'] = df_paris_flow['flow'] / df_paris_flow['length']
#get a probability map per tile and for mean of transports
df_paris_flow = df_paris_flow.groupby(['detid', 'day', 'hour', 'meanoftransport']).mean().reset_index()

In [9]:
# API endpoint
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/plan-de-voirie-chaussees/records"

# Fetch data from the API
response = requests.get(url)
data = response.json()

# Extract records
records = data['results']

# Normalize the data into a DataFrame
df = pd.json_normalize(records)

# Extract the 'geo_point_2d' field for coordinates
df['latitude'] = df['geo_point_2d.lat']
df['longitude'] = df['geo_point_2d.lon']

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']))

# # Save the DataFrame as CSV
# csv_path = 'paris_voirie_chaussees.csv'
# df.to_csv(csv_path, index=False)

# # Save the GeoDataFrame as GeoJSON
# geojson_path = 'paris_voirie_chaussees.geojson'
# gdf.to_file(geojson_path, driver='GeoJSON')

# print(f"Data saved as {csv_path} and {geojson_path}")


In [17]:
#paris
url_set = {
    'paves': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/plan-de-voirie-paves-mosaiques-du-plan-de-voirie-de-paris/records',
    'mobility_barometer': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/observatoire-parisien-des-mobilites-barometre-trimestriel-des-deplacements/records',
    'traffic_composition': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/compositions-du-trafic/records',
    'road_map': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/plan-de-voirie-emplacements-plaques-adresses/records',
    'roadways': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/plan-de-voirie-chaussees/records',
    'railways': 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/plan-de-voirie-emprises-ferroviaires/records',
}

In [15]:
data

{'total_count': 20956,
 'results': [{'objectid': 21,
   'num_pave': '051A',
   'igds_level': 35,
   'lib_level': 'RAMPES, CHAUSSEE',
   'lib_classe': 'Chaussée',
   'geo_shape': {'type': 'Feature',
    'geometry': {'coordinates': [[[2.3517077338929706, 48.84769933764409],
       [2.3516689130467587, 48.84770976669032],
       [2.351731494004273, 48.847810188196924],
       [2.351889853854827, 48.847766662825734],
       [2.351924818410569, 48.847757317231476],
       [2.3518161863137657, 48.84766991996834],
       [2.3517936764133163, 48.84767612409994],
       [2.3517077338929706, 48.84769933764409]]],
     'type': 'Polygon'},
    'properties': {}},
   'geo_point_2d': {'lon': 2.3517871926912814, 'lat': 48.847738933530756}},
  {'objectid': 30,
   'num_pave': '051A',
   'igds_level': 35,
   'lib_level': 'RAMPES, CHAUSSEE',
   'lib_classe': 'Chaussée',
   'geo_shape': {'type': 'Feature',
    'geometry': {'coordinates': [[[2.3554305103149114, 48.84881720991495],
       [2.3553303808730304

In [16]:
df

,objectid,num_pave,igds_level,lib_level,lib_classe,geo_shape.type,geo_shape.geometry.coordinates,geo_shape.geometry.type,geo_point_2d.lon,geo_point_2d.lat,latitude,longitude
0,21,051A,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3517077338929706, 48.84769933764409], [2....",Polygon,2.351787,48.847739,48.847739,2.351787
1,30,051A,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3554305103149114, 48.84881720991495], [2....",Polygon,2.355547,48.848962,48.848962,2.355547
2,32,144F,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3120274879761724, 48.82493253794935], [2....",Polygon,2.312463,48.824820,48.824820,2.312463
3,33,144F,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3101996264234494, 48.825588382697866], [2...",Polygon,2.310334,48.825725,48.825725,2.310334
4,43,191E,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.376260987532558, 48.88518172397364], [2.3...",Polygon,2.376496,48.884935,48.884935,2.376496
5,62,191E,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3800024099257517, 48.88472889187787], [2....",Polygon,2.379765,48.884747,48.884747,2.379765
6,83,191E,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.3809155696710804, 48.885555087735646], [2...",Polygon,2.381480,48.885732,48.885732,2.381480
7,120,144F,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.31391201628338, 48.82530558473369], [2.31...",Polygon,2.313909,48.825371,48.825371,2.313909
8,156,144F,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.308357024171957, 48.82369329548348], [2.3...",Polygon,2.308493,48.823610,48.823610,2.308493
9,171,161F,35,"RAMPES, CHAUSSEE",Chaussée,Feature,"[[[2.2648264070022193, 48.84843607377918], [2....",Polygon,2.264819,48.848606,48.848606,2.264819


In [18]:
def get_paris_data(url):
    response = requests.get(url)
    data = response.json()
    records = data['results']
    df = pd.json_normalize(records)
    return df

In [19]:
datasets = {}
for key, value in url_set.items():
    datasets[key] = get_paris_data(value)
    print(f"Data for {key} fetched. Shape: {df.shape}")

Data for paves fetched. Shape: (10, 12)
Data for mobility_barometer fetched. Shape: (10, 12)
Data for traffic_composition fetched. Shape: (10, 12)
Data for road_map fetched. Shape: (10, 12)
Data for roadways fetched. Shape: (10, 12)
Data for railways fetched. Shape: (10, 12)


In [20]:
df = datasets['traffic_composition']

In [21]:
df

,annee,site,troncon,arrondissement,latitude,longitude,periode,heure,vehicules,nombre,secteur,coordonnees.lon,coordonnees.lat
0,2014-01-01,Av. de la Grande Armée,de rue Le Sueur vers Rue Brunel,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,18h-19h,Bus,4.000000,Paris-Intra-Muros,2.28822,48.87624
1,2014-01-01,Av. de la Grande Armée,de rue Le Sueur vers Rue Brunel,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,18h-19h,Vélos Personnels,13.666667,Paris-Intra-Muros,2.28822,48.87624
2,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,09h-10h,Vélib's,9.000000,Paris-Intra-Muros,2.28822,48.87624
3,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,13h-14h,Poids lourds,33.000000,Paris-Intra-Muros,2.28822,48.87624
4,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,14h-15h,Bus,4.000000,Paris-Intra-Muros,2.28822,48.87624
5,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,16h-17h,Cars,8.000000,Paris-Intra-Muros,2.28822,48.87624
6,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,16h-17h,Deux roues-motorisés,480.666667,Paris-Intra-Muros,2.28822,48.87624
7,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,17h-18h,Véhicules Particuliers,1366.000000,Paris-Intra-Muros,2.28822,48.87624
8,2014-01-01,Av. de la Grande Armée,de Rue Brunel vers rue Le Sueur,75017,48.87624,2.28822,MOYENNE DU 18 AU 20 NOVEMBRE 2014,20h-21h,Véhicules Particuliers,1383.000000,Paris-Intra-Muros,2.28822,48.87624
9,2014-01-01,Porte de la Chapelle,de rue du Pré vers Rond Point de la Chapelle,BP,48.89882,2.35903,MOYENNE DU 18 AU 20 NOVEMBRE 2014,07h-08h,Véhicules Utilitaires,413.000000,Paris-Intra-Muros,2.35903,48.89882


considering the given datasets, build a normalized dataset for the city of Paris:
Tile Prob Map:
- tile_id
- latitude
- longitude
- tile_flow
- prob = tile_flow / sum_tile_flow

Mean of transports prob map:
- tile_id
- walk/byclies (each with a prob)
- vehicles 
- metro
- latitude
- longitude

- fit with the trajectory model